In [15]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib

import pandas as pd
import numpy as np
import seaborn as sns
import itertools

import sys
sys.path.append('../data/tecantaloupe/')

from growth.plate_spec import PlateSpec
from growth.plate_time_course_parser import SavageLabM1000Excel
from matplotlib import pyplot as plt

import seaborn as sns
from utils import *

matplotlib.rcParams.update({'font.size': 30})

In [16]:
allPfams = pd.read_csv('../data/coocurance/ALL_PFAM.txt',sep='\t')
allPfams

,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
0,A0A0Q7I8V6,A0A0Q7I9D6,PF00004,1,#FF0000,13,complement-normal
1,A0A248K2M4,A0A248K2S2,PF00004,1,#FF0000,17,complement-complement
2,A0A074JF46,A0A074JBY5,PF00004,1,#FF0000,8,normal-normal
3,A0A2N6DRE4,A0A2N6DRF5,PF00004,1,#FF0000,9,complement-normal
4,A0A1Y0NE98,A0A1Y0NDR2,PF00004,1,#FF0000,19,normal-complement
5,D4GWQ7,D4GSN1,PF00004,1,#FF0000,17,complement-normal
6,C7RLG2,C7RLH0,PF00004,1,#FF0000,8,normal-normal
7,C1DDM9,C1DDP9,PF00004,1,#FF0000,20,normal-complement
8,A0A1G5HPV2,A0A1G5HP54,PF00004-PF01434,1,#FF0000,17,normal-normal
9,A0A1G5HPV2,A0A1G5HP54,PF00004-PF01434,1,#FF0000,17,normal-normal


In [17]:
hasdabB = allPfams[allPfams['Neighbor Pfam'].str.contains('PF00361')]
print(len(hasdabB))
print(len(hasdabB['Query ID'].unique()))
posOperons = hasdabB[(hasdabB['Query-Neighbor Distance']<=3) & 
                 ((hasdabB['Query-Neighbor Directions'] == "normal-normal") 
                  | (hasdabB['Query-Neighbor Directions'] == 'complement-complement'))]['Query ID'].unique()
print(len(posOperons))
hasdabB

1734
840
835


,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
8113,A0A0Q6RAM1,A0A0Q6RMA6,PF00361,1,#FF0000,1,normal-normal
8114,A0A0U5B7A7,A0A0U5BBV8,PF00361,1,#FF0000,1,normal-normal
8115,A0A0R0LHR2,A0A0R0LH82,PF00361,1,#FF0000,1,normal-normal
8116,A0A149VTP0,A0A149VUJ7,PF00361,1,#FF0000,1,normal-normal
8117,A0A0W1SUY6,A0A0W1SUX4,PF00361,1,#FF0000,1,normal-normal
8118,A0A241R5F6,A0A241R6M8,PF00361,1,#FF0000,1,complement-complement
8119,A0A2S0III6,A0A2S0III1,PF00361,1,#FF0000,1,normal-normal
8120,A3WMF2,A3WMF1,PF00361,1,#FF0000,1,normal-normal
8121,A0A0F3K8W1,A0A0F3K915,PF00361,1,#FF0000,1,normal-normal
8122,G2J578,G2J577,PF00361,1,#FF0000,1,normal-normal


In [33]:
hasRub = allPfams[allPfams['Neighbor Pfam'].str.contains('PF00016') | allPfams['Neighbor Pfam'].str.contains('PF02788')]
print(len(hasRub))
print(len(hasRub['Query ID'].unique()))

hasRub

95
45


,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
1287,C6HYI8,C6HYJ2,PF00016,1,#FF0000,4,normal-normal
1288,A0A0R0LHR2,A0A0R0LRY0,PF00016-PF02788,1,#FF0000,17,normal-complement
1289,A0A149VTP0,A0A149VTQ8,PF00016-PF02788,1,#FF0000,17,normal-complement
1290,Q31HC3,Q31HD9,PF00016-PF02788,1,#FF0000,16,normal-normal
1291,A0A1N6DK18,A0A1N6DK16,PF00016-PF02788,1,#FF0000,4,normal-normal
1292,A0A1N6DK18,A0A1N6DJZ7,PF00016-PF02788,1,#FF0000,1,normal-normal
1293,A0A0P9EA54,A0A0P9C2Q3,PF00016-PF02788,1,#FF0000,6,complement-normal
1294,A0A0P9EA54,A0A0P9CK02,PF00016-PF02788,1,#FF0000,12,complement-complement
1295,A4YQE4,A4YQD3,PF00016-PF02788,1,#FF0000,11,normal-normal
1296,A0A149W153,A0A149W114,PF00016-PF02788,1,#FF0000,18,complement-normal


In [23]:
pfams = set()
for pfamG in allPfams['Neighbor Pfam']:
    pfamL = pfamG.split(sep='-')
    for pfam in pfamL:
        pfams.add(pfam)
len(pfams)

4289

In [28]:
alreadyMade=True
if alreadyMade == False:
    pfam_df = pd.DataFrame(columns = ['pfam','occurances','uniqueGenomes'])
    pfamCol = []
    occCol = []
    uniCol = []
    for pfam in pfams:
        has = allPfams[allPfams['Neighbor Pfam'].str.contains(pfam)]
        pfamCol.append(pfam)
        occCol.append(len(has))
        uniCol.append(len(has['Query ID'].unique()))
    pfam_df['pfam']=pfamCol
    pfam_df['occurances']=occCol
    pfam_df['uniqueGenomes']=uniCol
    pfam_df.to_csv('../data/coocurance/pfam_occurances.csv')
else:
    pfam_df = pd.read_csv('../data/coocurance/pfam_occurances.csv')
    alreadyMade=True
    

In [32]:
pfam_df.sort_values('uniqueGenomes',ascending=False)

,Unnamed: 0,pfam,occurances,uniqueGenomes
829,829,none,6292,859
3268,3268,PF00361,1734,840
2542,2542,PF00662,1264,604
1810,1810,PF03466,1262,465
3081,3081,PF00126,1256,461
2442,2442,PF00005,715,328
3378,3378,PF00072,1068,296
3162,3162,PF02518,1020,269
1596,1596,PF00512,842,215
2549,2549,PF00528,299,143
